<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/S2bgrnExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth 
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize() # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

# Some common imports
from IPython.display import Image
import folium

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=FavRjU9DGTBrWAaybnsCw6NtsKxV9H8JHOJ4UiQf_tw&tc=MPWWOlQTfaql3upEco8jywDPQ4v6-zYln8Aa0yqlIRo&cc=Jbv3Y5c1N0Q4h3cW08o7Pv95_7ZwqeWVo6o4AUAgasQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJpK_mYCOmqajGv2xzGm5XV0iQUYps4zGCqUZznqKPxRkS67mrTFNF8

Successfully saved authorization token.
Mounted at /content/drive


In [ ]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

# Get percentile cover   
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actArea = ee.Number(image.updateMask(image.select('B2')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      maxPixels=1e12,
      ).values().get(0)).multiply(10000)
  # calculate the perc of cover OF CLEAR PIXELS 
  percCover = actArea.divide(area).multiply(100)
  # number as output
  return image.set('percCover', percCover,'actArea',actArea)

# Select bgrn bands
def selectBands(image):
  img = image.select(['B2', 'B3', 'B4', 'B8'])
  return img

def getDims(image):
  dimensions = img.select('B2').getInfo().get('bands')[0].get('dimensions')
  return image.set('dims', dimensions)

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Original method...best image in a month
tiles = ee.FeatureCollection('projects/ee-eric-levenson/assets/ArcticAKaois/S2Tiles_AK_7N')
tile = '07WDS'
roi = tiles.filter(ee.Filter.eq('Name', tile)).first()
start = '2016-05-15'
finish = '2016-09-30'
startMonth = 5
finishMonth = 5
eestart = ee.Date(start)
eefinish = ee.Date(finish)
coverage = 90 # lowest clear roi percentage threshold
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi.geometry()).filter(ee.Filter.equals('MGRS_TILE',tile)).filterDate(start,finish).filter(ee.Filter.calendarRange(startMonth, finishMonth, 'month')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = images.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(clip_image) # Clip to roi
images_all = images_all.map(applyMask) # mask other bands that are cloudy
area = roi.geometry().area().getInfo()
images_all = images_all.map(getCover) # Add percent cover as an image property
images_all = images_all.filterMetadata('percCover','greater_than', coverage) # remove images covering less than 50% of the ROI)
images_all = images_all.map(selectBands) # reduce to bgrn bands
percCovers = images_all.aggregate_array('percCover').getInfo() # get non EE list of percent covers
maxim = max(percCovers) # get maximum percent cover
best_image = images_all.filterMetadata('percCover','equals',maxim) # get image with max percent cover
img = ee.Image(best_image.first())
date = img.getInfo()['id'].split('/')[2].split('_')[0][:8] # get date for export
filename = str(date+'_'+tile+'_BGRN_SR')

In [ ]:
percCovers

[100.2532169419131, 100.2532169419131]

In [ ]:
filename

'20160530_07WDS_BGRN_SR'

In [ ]:
## Visualize to double check ##

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B8', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': [0.95, 1.1, 1]
}
# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(zoom_start=10)
# Add the image layer to the map and display it.
map_l8.add_ee_layer(img, image_viz_params, 'true color composite')
#map_l8.add_ee_layer(image_mask, {'bands': ['B2']}, 'true color composite')
folium.GeoJson(roi.geometry().getInfo()).add_to(map_l8)
display(map_l8) 

In [ ]:
# Exporting asset to drive example (region could be specified)
task = ee.batch.Export.image.toDrive(**{
    'image': img,
    'description': filename,
    'folder':'S2_BGRN',
    'fileFormat': 'GeoTIFF',
    'scale': 10,
    'region': roi.geometry(),
    'maxPixels': 1e12
})
task.start()
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id: VYCHXLBVKRFCVWNFRVVQY6ZY).
Polling for task (id

In [ ]:
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32607',
   'crs_transform': [10, 0, 399960, 0, -10, 7800000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32607',
   'crs_transform': [10, 0, 399960, 0, -10, 7800000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32607',
   'crs_transform': [10, 0, 399960, 0, -10, 7800000]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32607',
   'crs_transform': [10, 0, 399960, 0, -10, 7800000]}],
 'id': 'COPERNICUS/S2/20180920T213521_20180920T213524_T07WDT',
 'version': 161770884